First set up code to visualize a sound form

In [1]:
%pylab inline
import IPython.display as ipd
import librosa
from librosa import load, display
import glob
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()
import pickle
from common import save_as_pickle
# import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np

In [3]:
# You should change these paths according to the path of the files on your system.
PATH_TO_TRAIN_LABELS = "data/train/train.csv"
PATH_TO_TEST_LABELS = "data/test/test.csv"
PATH_TO_TRAIN_AUDIO_FILES = "data/train/wav/"
PATH_TO_TEST_AUDIO_FILES = "data/test/wav/"
PATH_TO_SUBMISSION = "submission/"
PATH_TO_PICKLE = "pickles/"
SUBMISSION_TITLE = "nn contrast"

In [4]:
# It is easier to deal with csv if you can load it into a structure you can work with.
# Pandas are the most convenient way to do that and are available with 
# inbuilt functionality to handle csv file.

# Pandas assumes that the first row in your file is the header adn not the actual values.
# This behavior can be overriden by passing header=None as a parameter.
train = pd.read_csv(PATH_TO_TRAIN_LABELS)
test = pd.read_csv(PATH_TO_TEST_LABELS)

In [5]:
# You can reactivate this cell to make sure your model is working correctly in terms of dimensions.
#train = train[:2]
#test = test[:2]

In [6]:
train_error_count = 0
train_error_labels = []
test_error_count = 0
test_error_labels = []

In [7]:
# To start with classification, we first need to convert the wav sound files into a format we can work 
# with. It is easier to take the amplitude at each sampling point and use that 
# numeric value to form a feature vector.
def train_parser(row):
    global train_error_count
    global train_error_labels
    path_to_wav_files = PATH_TO_TRAIN_AUDIO_FILES
    file_path = path_to_wav_files + str(row.ID) + ".wav"
    try:
        data, sampling_rate = librosa.load(file_path)
        stft = np.abs(librosa.stft(data))
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sampling_rate).T,axis=0)
    except Exception as ex:
        print(ex)
        train_error_count += 1
        train_error_labels.append(row.ID)
        return [0]*7, row.Class
    features = contrast
    label = row.Class
    return [features, label]

In [8]:
# To create the training feature matrix, we can apply our parser to each training sample.
train_features = train.progress_apply(train_parser,axis=1)
print("%d samples had errors while parsing" % train_error_count)
print("Errorneous samples", train_error_labels)
save_as_pickle(data=train_features,pickle_file=PATH_TO_PICKLE + SUBMISSION_TITLE + " train.pickle" )


0 samples had errors while parsing
Errorneous samples []


In [9]:
# Renaming the columns to singnify what they mean helps with documentation,
# and also helps you keep track of them later on.
train_features.columns = ['feature','label']
# train_features.head()

In [10]:
# this library helps us convert string labels into easy to handle encoded labels.
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
X = np.array(train_features.feature.tolist())
Y = np.array(train_features.label.tolist())
lb = LabelEncoder()
# Since labels are categories they dont inherently have an order amongst themselves.
# For example, Apples > oranges does not make any sense. So to madel such categorical 
# variables, we can convert them to one hot vectors.
Y = to_categorical(lb.fit_transform(Y))

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [13]:
number_of_labels = Y.shape[1]
filter_size = 2

In [14]:
model = Sequential()

In [15]:
model.add(Dense(256, input_shape=(7,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [16]:
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [17]:
model.add(Dense(number_of_labels))
model.add(Activation('softmax'))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               2048      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
__________

In [19]:
model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='adam')


In [20]:
model.fit(X,Y, batch_size=32, epochs=50)

Epoch 1/50
5435/5435 [==============================] - ETA: 1:35 - loss: 8.9289 - acc: 0.0000e+0 - ETA: 12s - loss: 6.9537 - acc: 0.1367    - ETA: 6s - loss: 6.6928 - acc: 0.1208 - ETA: 4s - loss: 6.1828 - acc: 0.131 - ETA: 3s - loss: 5.6485 - acc: 0.127 - ETA: 2s - loss: 5.1909 - acc: 0.127 - ETA: 2s - loss: 4.9400 - acc: 0.127 - ETA: 2s - loss: 4.6570 - acc: 0.129 - ETA: 1s - loss: 4.3951 - acc: 0.129 - ETA: 1s - loss: 4.1674 - acc: 0.128 - ETA: 1s - loss: 4.0465 - acc: 0.128 - ETA: 1s - loss: 3.9030 - acc: 0.130 - ETA: 1s - loss: 3.7685 - acc: 0.132 - ETA: 1s - loss: 3.6630 - acc: 0.134 - ETA: 0s - loss: 3.5555 - acc: 0.133 - ETA: 0s - loss: 3.4597 - acc: 0.137 - ETA: 0s - loss: 3.3721 - acc: 0.140 - ETA: 0s - loss: 3.3655 - acc: 0.140 - ETA: 0s - loss: 3.2923 - acc: 0.145 - ETA: 0s - loss: 3.2378 - acc: 0.146 - ETA: 0s - loss: 3.1838 - acc: 0.148 - ETA: 0s - loss: 3.1337 - acc: 0.150 - ETA: 0s - loss: 3.0999 - acc: 0.152 - ETA: 0s - loss: 3.0662 - acc: 0.154 - 2s 373us/step - loss

5435/5435 [==============================] - ETA: 1s - loss: 2.0541 - acc: 0.312 - ETA: 1s - loss: 1.8840 - acc: 0.363 - ETA: 1s - loss: 1.7707 - acc: 0.391 - ETA: 1s - loss: 1.7500 - acc: 0.383 - ETA: 1s - loss: 1.7452 - acc: 0.379 - ETA: 0s - loss: 1.7664 - acc: 0.370 - ETA: 0s - loss: 1.7826 - acc: 0.359 - ETA: 0s - loss: 1.7877 - acc: 0.351 - ETA: 0s - loss: 1.7918 - acc: 0.347 - ETA: 0s - loss: 1.8010 - acc: 0.342 - ETA: 0s - loss: 1.8013 - acc: 0.342 - ETA: 0s - loss: 1.8063 - acc: 0.339 - ETA: 0s - loss: 1.8126 - acc: 0.337 - ETA: 0s - loss: 1.8126 - acc: 0.338 - ETA: 0s - loss: 1.8201 - acc: 0.335 - ETA: 0s - loss: 1.8165 - acc: 0.336 - ETA: 0s - loss: 1.8168 - acc: 0.338 - ETA: 0s - loss: 1.8103 - acc: 0.337 - ETA: 0s - loss: 1.8096 - acc: 0.339 - ETA: 0s - loss: 1.8087 - acc: 0.338 - ETA: 0s - loss: 1.8043 - acc: 0.341 - ETA: 0s - loss: 1.8042 - acc: 0.342 - ETA: 0s - loss: 1.8043 - acc: 0.341 - ETA: 0s - loss: 1.8084 - acc: 0.339 - 1s 244us/step - loss: 1.8095 - acc: 0.3395


5435/5435 [==============================] - ETA: 3s - loss: 1.6143 - acc: 0.437 - ETA: 1s - loss: 1.6967 - acc: 0.392 - ETA: 1s - loss: 1.6944 - acc: 0.394 - ETA: 1s - loss: 1.7056 - acc: 0.394 - ETA: 1s - loss: 1.7136 - acc: 0.389 - ETA: 1s - loss: 1.7107 - acc: 0.389 - ETA: 1s - loss: 1.7179 - acc: 0.383 - ETA: 1s - loss: 1.7266 - acc: 0.382 - ETA: 1s - loss: 1.7410 - acc: 0.375 - ETA: 0s - loss: 1.7401 - acc: 0.372 - ETA: 0s - loss: 1.7416 - acc: 0.371 - ETA: 0s - loss: 1.7364 - acc: 0.370 - ETA: 0s - loss: 1.7319 - acc: 0.368 - ETA: 0s - loss: 1.7340 - acc: 0.368 - ETA: 0s - loss: 1.7361 - acc: 0.368 - ETA: 0s - loss: 1.7282 - acc: 0.372 - ETA: 0s - loss: 1.7193 - acc: 0.374 - ETA: 0s - loss: 1.7168 - acc: 0.374 - ETA: 0s - loss: 1.7127 - acc: 0.377 - ETA: 0s - loss: 1.7129 - acc: 0.374 - ETA: 0s - loss: 1.7159 - acc: 0.371 - ETA: 0s - loss: 1.7142 - acc: 0.373 - ETA: 0s - loss: 1.7184 - acc: 0.371 - 1s 251us/step - loss: 1.7187 - acc: 0.3726
Epoch 18/50
5435/5435 [===============

5435/5435 [==============================] - ETA: 2s - loss: 1.6729 - acc: 0.312 - ETA: 1s - loss: 1.6472 - acc: 0.386 - ETA: 1s - loss: 1.6272 - acc: 0.396 - ETA: 1s - loss: 1.6287 - acc: 0.407 - ETA: 1s - loss: 1.6229 - acc: 0.418 - ETA: 1s - loss: 1.6387 - acc: 0.413 - ETA: 0s - loss: 1.6361 - acc: 0.416 - ETA: 0s - loss: 1.6544 - acc: 0.410 - ETA: 0s - loss: 1.6517 - acc: 0.414 - ETA: 0s - loss: 1.6491 - acc: 0.413 - ETA: 0s - loss: 1.6459 - acc: 0.413 - ETA: 0s - loss: 1.6499 - acc: 0.411 - ETA: 0s - loss: 1.6510 - acc: 0.407 - ETA: 0s - loss: 1.6540 - acc: 0.403 - ETA: 0s - loss: 1.6593 - acc: 0.399 - ETA: 0s - loss: 1.6619 - acc: 0.398 - ETA: 0s - loss: 1.6628 - acc: 0.396 - ETA: 0s - loss: 1.6663 - acc: 0.395 - ETA: 0s - loss: 1.6647 - acc: 0.395 - ETA: 0s - loss: 1.6618 - acc: 0.396 - ETA: 0s - loss: 1.6659 - acc: 0.395 - ETA: 0s - loss: 1.6674 - acc: 0.396 - ETA: 0s - loss: 1.6683 - acc: 0.395 - 1s 243us/step - loss: 1.6664 - acc: 0.3961
Epoch 26/50
5435/5435 [===============

5435/5435 [==============================] - ETA: 0s - loss: 1.7195 - acc: 0.406 - ETA: 1s - loss: 1.5758 - acc: 0.399 - ETA: 1s - loss: 1.5735 - acc: 0.404 - ETA: 1s - loss: 1.5717 - acc: 0.410 - ETA: 1s - loss: 1.5940 - acc: 0.404 - ETA: 0s - loss: 1.6039 - acc: 0.407 - ETA: 0s - loss: 1.5929 - acc: 0.422 - ETA: 0s - loss: 1.5947 - acc: 0.422 - ETA: 0s - loss: 1.6038 - acc: 0.413 - ETA: 0s - loss: 1.6085 - acc: 0.413 - ETA: 0s - loss: 1.6097 - acc: 0.408 - ETA: 0s - loss: 1.5949 - acc: 0.414 - ETA: 0s - loss: 1.5954 - acc: 0.415 - ETA: 0s - loss: 1.5936 - acc: 0.418 - ETA: 0s - loss: 1.5949 - acc: 0.417 - ETA: 0s - loss: 1.6058 - acc: 0.415 - ETA: 0s - loss: 1.6060 - acc: 0.413 - ETA: 0s - loss: 1.6065 - acc: 0.412 - ETA: 0s - loss: 1.6070 - acc: 0.413 - ETA: 0s - loss: 1.6073 - acc: 0.411 - ETA: 0s - loss: 1.6086 - acc: 0.411 - ETA: 0s - loss: 1.6112 - acc: 0.410 - ETA: 0s - loss: 1.6077 - acc: 0.412 - ETA: 0s - loss: 1.6098 - acc: 0.411 - 1s 243us/step - loss: 1.6095 - acc: 0.4110


5435/5435 [==============================] - ETA: 0s - loss: 1.3977 - acc: 0.406 - ETA: 1s - loss: 1.4728 - acc: 0.447 - ETA: 1s - loss: 1.5938 - acc: 0.415 - ETA: 1s - loss: 1.6211 - acc: 0.416 - ETA: 1s - loss: 1.6104 - acc: 0.431 - ETA: 1s - loss: 1.5965 - acc: 0.432 - ETA: 0s - loss: 1.5752 - acc: 0.434 - ETA: 0s - loss: 1.5953 - acc: 0.424 - ETA: 0s - loss: 1.5960 - acc: 0.423 - ETA: 0s - loss: 1.5952 - acc: 0.427 - ETA: 0s - loss: 1.5853 - acc: 0.429 - ETA: 0s - loss: 1.5895 - acc: 0.429 - ETA: 0s - loss: 1.5838 - acc: 0.425 - ETA: 0s - loss: 1.5843 - acc: 0.420 - ETA: 0s - loss: 1.5829 - acc: 0.424 - ETA: 0s - loss: 1.5847 - acc: 0.424 - ETA: 0s - loss: 1.5856 - acc: 0.425 - ETA: 0s - loss: 1.5875 - acc: 0.422 - ETA: 0s - loss: 1.5893 - acc: 0.422 - ETA: 0s - loss: 1.5881 - acc: 0.421 - ETA: 0s - loss: 1.5882 - acc: 0.421 - ETA: 0s - loss: 1.5896 - acc: 0.420 - ETA: 0s - loss: 1.5874 - acc: 0.420 - 1s 242us/step - loss: 1.5874 - acc: 0.4204
Epoch 42/50
5435/5435 [===============

5435/5435 [==============================] - ETA: 1s - loss: 1.7049 - acc: 0.375 - ETA: 1s - loss: 1.5169 - acc: 0.429 - ETA: 1s - loss: 1.5032 - acc: 0.447 - ETA: 1s - loss: 1.5397 - acc: 0.436 - ETA: 1s - loss: 1.5479 - acc: 0.437 - ETA: 1s - loss: 1.5464 - acc: 0.440 - ETA: 1s - loss: 1.5519 - acc: 0.434 - ETA: 0s - loss: 1.5556 - acc: 0.437 - ETA: 0s - loss: 1.5544 - acc: 0.446 - ETA: 0s - loss: 1.5492 - acc: 0.445 - ETA: 0s - loss: 1.5562 - acc: 0.444 - ETA: 0s - loss: 1.5530 - acc: 0.447 - ETA: 0s - loss: 1.5526 - acc: 0.444 - ETA: 0s - loss: 1.5603 - acc: 0.440 - ETA: 0s - loss: 1.5588 - acc: 0.437 - ETA: 0s - loss: 1.5535 - acc: 0.436 - ETA: 0s - loss: 1.5555 - acc: 0.432 - ETA: 0s - loss: 1.5536 - acc: 0.432 - ETA: 0s - loss: 1.5462 - acc: 0.435 - ETA: 0s - loss: 1.5389 - acc: 0.439 - ETA: 0s - loss: 1.5330 - acc: 0.441 - ETA: 0s - loss: 1.5340 - acc: 0.442 - ETA: 0s - loss: 1.5349 - acc: 0.440 - 1s 240us/step - loss: 1.5323 - acc: 0.4414
Epoch 50/50
5435/5435 [===============

In [21]:
def test_parser(row):
    global test_error_count
    global test_error_labels
    path_to_wav_files = PATH_TO_TEST_AUDIO_FILES
    file_path = path_to_wav_files + str(row.ID) + ".wav"
    try:
        data, sampling_rate = librosa.load(file_path)
        stft = np.abs(librosa.stft(data))
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sampling_rate).T,axis=0)
    except Exception as ex:
        test_error_count += 1
        test_error_labels.append(row.ID)
        return pd.Series([0]*7)
    features = contrast
    return pd.Series(features)

In [22]:
test_features = test.progress_apply(test_parser,axis=1, reduce = True)
print("%d samples had errors while parsing" % test_error_count)
print("Errorneous samples", test_error_labels)
save_as_pickle(data=train_features,pickle_file=PATH_TO_PICKLE + SUBMISSION_TITLE + " test.pickle" )


0 samples had errors while parsing
Errorneous samples []


In [23]:
X_test = test_features
# X_test

In [24]:
test_labels = model.predict(X_test, batch_size=32)

In [25]:
test_labels_strings = lb.inverse_transform(test_labels.argmax(axis=1))
# test_labels_strings

C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [26]:
test['Class'] = test_labels_strings

In [27]:
test.to_csv(PATH_TO_SUBMISSION + SUBMISSION_TITLE + ".csv",index=None)

This approach gives 56% accuracy with the above setup.